In [ ]:
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adamax
from pathlib import Path
from sklearn.metrics import ConfusionMatrixDisplay

from python import classes, metrics, data_loader

In [ ]:
notebook_classification = ""
notebook_cv = 0

image_size = 112
batch_size = 32
total_cv = 5

max_epochs_1 = 15
lr_scheduler_patience_1 = 3
early_stopping_patience_1 = 5
max_epochs_2 = 50
lr_scheduler_patience_2 = 3
early_stopping_patience_2 = 5

data_dir = "data"
tensorboard_dir = "out/logs"
metrics_dir = "out/metrics"
models_dir = "out/keras"
weights_dir = "out/weights"
gpu_memory_limit = 1024*8

In [ ]:
assert notebook_classification in ['models', 'types'], "notebook_classification must be one of ['models', 'types']"
assert notebook_cv != 0, "notebook_cv must be provided"
assert 1 <= notebook_cv <= total_cv, "notebook_cv must be in the range [1, total_cv]"

In [ ]:
class_names = classes.class_names[notebook_classification]
classes_num = len(class_names)

notebook_model = "m3"
data_dir = Path(data_dir) / f"{notebook_classification}"
tensorboard_dir = Path(tensorboard_dir) / f"{notebook_model}/{notebook_classification}/cv{notebook_cv}"
metrics_file = Path(metrics_dir) / f"{notebook_model}/{notebook_classification}.json"
model_file = Path(models_dir) / f"{notebook_model}/{notebook_classification}/cv{notebook_cv}.keras"
weights_file = Path(weights_dir) / f"{notebook_model}/{notebook_classification}/cv{notebook_cv}.weights.h5"

In [ ]:
assert not model_file.is_file(), "This model already exists"

In [ ]:
shutil.rmtree(tensorboard_dir, ignore_errors=True)

tensorboard_dir.mkdir(parents=True, exist_ok=True)
metrics_file.parent.mkdir(parents=True, exist_ok=True)
model_file.parent.mkdir(parents=True, exist_ok=True)
weights_file.parent.mkdir(parents=True, exist_ok=True)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if len(gpus) > 0:
    tf.config.set_logical_device_configuration(gpus[0], [
        tf.config.LogicalDeviceConfiguration(memory_limit=gpu_memory_limit)
    ])

In [ ]:
def gen_metrics():
    return [
        'accuracy', 
        metrics.F1Macro(classes_num), 
        metrics.PrecisionMacro(classes_num),
        metrics.RecallMacro(classes_num)
    ]

def gen_callbacks(lr_patience, es_patience):
    return [
        TensorBoard(log_dir=tensorboard_dir),
        ModelCheckpoint(weights_file, save_best_only=True, save_weights_only=True),
        ReduceLROnPlateau(monitor='val_loss', patience=lr_patience, min_lr=1e-6),
        EarlyStopping(monitor='val_loss', patience=es_patience)
    ]

In [ ]:
train_data, val_data = data_loader.load_data(
    data_dir=data_dir,
    val_fold=notebook_cv,
    total_folds=total_cv,
    class_names=class_names,
    batch_size=batch_size,
    image_size=image_size
)

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1)
])

In [ ]:
base_model = tf.keras.applications.efficientnet.EfficientNetB3(
    include_top=False,
    input_shape=(image_size, image_size, 3), 
    pooling='max'
)

In [ ]:
# Input layer
inputs = layers.Input(shape=(image_size, image_size, 3))
x = data_augmentation(inputs)

# Base model
x = base_model(x, training=False)

# Head model
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Output layer
outputs = layers.Dense(classes_num, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

In [ ]:
base_model.trainable = False
model.compile(
    optimizer=Adamax(learning_rate=0.001), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    metrics=gen_metrics()
)

In [ ]:
history = model.fit(
    train_data, epochs=max_epochs_1, validation_data=val_data, verbose=2,
    callbacks=gen_callbacks(lr_scheduler_patience_1, early_stopping_patience_1)
);

In [ ]:
model.load_weights(weights_file)
next_epoch = len(history.history['val_loss'])

In [ ]:
base_model.trainable = True
model.compile(
    optimizer=Adamax(learning_rate=0.0001), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    metrics=gen_metrics()
)

In [ ]:
model.fit(
    train_data, epochs=max_epochs_1+max_epochs_2, validation_data=val_data, initial_epoch=next_epoch, verbose=2,
    callbacks=gen_callbacks(lr_scheduler_patience_2, early_stopping_patience_2)
);

In [ ]:
model.load_weights(weights_file)
model.save(model_file)

In [ ]:
fold_metrics = metrics.evaluate_metrics(model, val_data)
metrics.save_metrics(fold_metrics, metrics_file, notebook_cv)
fold_metrics

In [ ]:
y_true, y_pred = metrics.model_predict(model, val_data)

fig, ax = plt.subplots(figsize=(classes_num, classes_num))
cmp = ConfusionMatrixDisplay.from_predictions(
    y_true, y_pred, 
    display_labels=class_names, 
    cmap=plt.get_cmap('Blues'),
    colorbar=False, 
    xticks_rotation='vertical', 
    ax=ax
)